In [1]:
import importlib
import modules
import cellpylib as cpl
import numpy as np
import matplotlib.pyplot as plt
import scipy

importlib.reload(modules)

from modules import run_model
from modules import CA


In [5]:
def func(x, a, b):
    return [a * x_i ** -b for x_i in x]

def single_fit(x, y):
    popt1, pcov1 = scipy.optimize.curve_fit(func, x, y, p0=[1, 0.5])
    a1, b1 = popt1

    return a1, b1
        
def double_fit(x, y):
    highest_b2 = 0

    for index in range(int(len(x)*0.2), int(len(x)*0.8)):
        popt1, pcov1 = scipy.optimize.curve_fit(func, x[0:index], y[0:index], p0=[1, 0.5])
        a1, b1 = popt1

        popt2, pcov2 = scipy.optimize.curve_fit(func, x[index:], y[index:], p0=[1, 0.5])
        a2, b2 = popt2

        if b2 > highest_b2:
            highest_b2 = b2

            best_a1, best_b1, best_a2, best_b2 = a1, b1, a2, b2

            optimal_index = index

    return best_a1, best_b1, best_a2, best_b2, optimal_index

def find_intersection(x_data, a1, b1, a2, b2):
    x = np.linspace(1, np.max(x_data), np.max(x_data))
    y1 = func(x, a1, b1)
    y2 = func(x, a2, b2)
    for index in range(len(x)):
        if y1[index] >= y2[index]:
            return x[index]
        
    return None

def plot_fit(x, y, a1, b1, a2, b2):
    plt.clf()
    plt.loglog(x, y, 'o', markersize=2)
    plt.xlim(0, 1.1*np.max(x))
    plt.ylim(0, 1.1*np.max(y))
    plt.loglog(x, func(x, a1, b1))
    plt.loglog(x, func(x, a2, b2))
    plt.show()

def find_critical_size(density_list, n_simulations, L, T, n_repetitions):
    critical_size_dict = {}
    for p in density_list:
        for n in range(n_simulations): 
            print(p, n)
            lifespan_counter, final_jams_counted = run_model(p, L, T, n_repetitions)
            lists = sorted(final_jams_counted.items()) # sorted by key, return a list of tuples

            x_data, y_data = zip(*lists)
            a_optimal1, b_optimal1, a_optimal2, b_optimal2, optimal_index = double_fit(x_data, y_data)

            critical_size = find_intersection(x_data, a_optimal1, b_optimal1, a_optimal2, b_optimal2)

            if critical_size == None:
                if p in critical_size_dict:
                    del critical_size_dict[p]        
                break

            else:
                if p not in critical_size_dict:
                    critical_size_dict[p] = [critical_size]
                else:
                    critical_size_dict[p] += [critical_size]

    return critical_size_dict
        

In [4]:
density_list = np.linspace(0.25, 0.75, 11)
n_simulations = 10
L=1000
T=int(L/2)
n_repetitions = 100

critical_size_dict = find_critical_size(density_list, n_simulations, L, T, n_repetitions)

Unexpected exception formatting exception. Falling back to standard exception


Traceback (most recent call last):
  File "/Users/jennadevries/anaconda3/lib/python3.10/site-packages/IPython/core/interactiveshell.py", line 3526, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "/var/folders/f8/8_w9799j2wlfkyxvlcmd3z180000gn/T/ipykernel_26218/3050793384.py", line 7, in <module>
    critical_size_dict = find_critical_size(density_list, n_simulations, L, T, n_repetitions)
  File "/var/folders/f8/8_w9799j2wlfkyxvlcmd3z180000gn/T/ipykernel_26218/532800878.py", line 52, in find_critical_size
    lifespan_counter, final_jams_counted = run_model(p, L, T, n_repetitions)
  File "/Users/jennadevries/Documents/GitHub/CSS-Group1/modules.py", line 247, in run_model
    cellular_automaton = triangulize_evolution(cellular_automaton)
  File "/Users/jennadevries/Documents/GitHub/CSS-Group1/modules.py", line -1, in triangulize_evolution
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  